In [1]:
import pandas as pd 
import numpy as np 

In [3]:
df1 = pd.read_csv("Google_stock_price_train.csv")
df2 = pd.read_csv("Google_stock_price_test.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [7]:
df1['Close'] = df1['Close'].astype(str).str.replace(",","").astype(float)
df2['Close'] = df2['Close'].astype(str).str.replace(",","").astype(float)

In [8]:
from sklearn.preprocessing import MinMaxScaler
train_scaler = MinMaxScaler()
df1['Normalized close'] = train_scaler.fit_transform(df1['Close'].values.reshape(-1,1))

In [9]:
test_scaler = MinMaxScaler()
df2['Normalized close'] = test_scaler.fit_transform(df2['Close'].values.reshape(-1,1))

In [10]:
x_train = df1['Normalized close'].values[:-1].reshape(-1,1,1)
y_train = df1['Normalized close'].values[1:].reshape(-1,1,1)

x_test = df2['Normalized close'].values[:-1].reshape(-1,1,1)
y_test = df2['Normalized close'].values[1:].reshape(-1,1,1)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(4, input_shape = (1,1)))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')

model.summary()

C:\Users\karan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(x_train,y_train, validation_data= (x_test,y_test), epochs = 100, batch_size= 1)

Epoch 1/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0643 - val_loss: 0.0275
Epoch 2/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0060 - val_loss: 0.0244
Epoch 3/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0022 - val_loss: 0.0250
Epoch 4/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.7183e-04 - val_loss: 0.0248
Epoch 5/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0017 - val_loss: 0.0252
Epoch 6/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4.4691e-04 - val_loss: 0.0249
Epoch 7/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 6.9592e-04 - val_loss: 0.0252
Epoch 8/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5.7684e-04 - val_loss: 0.0251
Epoch 9/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.0900e-04 - val_loss: 0.0251
Epoch 10/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.8187e-04 - val_loss: 0.0250
Epoch 11/100
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011 - val_loss: 0.

In [13]:
test_loss = model.evaluate(x_test,y_test)
print('Testing loss: ', test_loss)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step - loss: 0.0243
Testing loss:  0.02428968995809555


In [14]:
pred = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


In [15]:
y_test_actual = test_scaler.inverse_transform(y_test.reshape(-1,1))
y_test_pred = test_scaler.inverse_transform(pred.reshape(-1,1))

In [16]:
index = 1
print("Actual: ", y_test_actual[index])
print("Predicted: ", y_test_pred[index])

Actual:  [794.02]
Predicted:  [787.59625]
